In [15]:
import requests
import pytz
import json
from datetime import datetime, time as datetime_time, timedelta
import calendar
import pandas as pd


In [16]:
class BearerAuth(requests.auth.AuthBase):
    def __init__(self, token):
        self.token = token

    def __call__(self, r):
        r.headers["authorization"] = "Bearer " + self.token
        return r


In [17]:
def make_api_request(stop_id, bearer):
    open_data_api_endpoint = "https://opendata-api.stib-mivb.be/OperationMonitoring/4.0/PassingTimeByPoint/" + stop_id
    headers = {"Accept": "application/json"}

    response = requests.get(open_data_api_endpoint, auth=BearerAuth(bearer))
    
    return response.json()

In [18]:
def parse_waiting_times(json_payload, destination):
    passing_times = json_payload['points'][0]['passingTimes']
    str_expected_arrival_times = [passing_time['expectedArrivalTime'] for passing_time in passing_times if passing_time['destination']['fr']==destination]
    print(str_expected_arrival_times)
    expected_arrival_times = [datetime.fromisoformat(str_expected_arrival_time) for str_expected_arrival_time in str_expected_arrival_times]
    return expected_arrival_times 

In [19]:
def compute_time_diff(start, end):
    if isinstance(start, datetime_time): # convert to datetime
        assert isinstance(end, datetime_time)
        start, end = [datetime.combine(datetime.min, t) for t in [start, end]]
    if start <= end: # e.g., 10:33:26-11:15:49
        return end - start
    else: # end < start e.g., 23:55:00-00:25:00
        end += timedelta(1) # +day
        assert end > start
        return end - start

In [20]:
from datetime import datetime, time as datetime_time, timedelta
import pytz


class TimeUtils:

    @staticmethod
    def get_current_localized_time():
        current_time_utc = datetime.utcnow()
        return pytz.utc.localize(current_time_utc)

    @staticmethod
    def compute_time_diff(start, end):
        if isinstance(start, datetime_time): # convert to datetime
            assert isinstance(end, datetime_time)
            start, end = [datetime.combine(datetime.min, t) for t in [start, end]]
        if start <= end: # e.g., 10:33:26-11:15:49
            return TimeUtils._convert_timedelta(end - start)
        else: # end < start e.g., 23:55:00-00:25:00
            end += timedelta(1) # +day
            assert end > start
            return TimeUtils._convert_timedelta(end - start)

    @staticmethod
    def _convert_timedelta(duration):
        duration_dict = {}
        days, seconds = duration.days, duration.seconds
        duration_dict['days'] = days
        hours = seconds // 3600
        minutes = (seconds % 3600) // 60
        seconds = (seconds % 60)
        duration_dict['hours'] = hours
        duration_dict['minutes'] = minutes
        duration_dict['seconds'] = seconds

        return duration_dict


In [21]:
from dataclasses import dataclass, field
from typing import List, Optional, Any

from dataclasses_json import dataclass_json, config, LetterCase

from datetime import datetime

@dataclass_json(letter_case=LetterCase.CAMEL)
@dataclass
class Message:
    fr: str = ""
    nl: str = ""
        
@dataclass_json(letter_case=LetterCase.CAMEL)
@dataclass
class Destination:
    fr: str = ""
    nl: str = ""

@dataclass_json(letter_case=LetterCase.CAMEL)
@dataclass
class PassingTime:
    destination: Optional[Destination] = None
    message: Optional[Message] = None
    line_id: str = ""
    expected_arrival_time: str = ""
    arriving_in_dict: dict = field(init=False, repr=False)
        
    def __post_init__(self):
        self.expected_arrival_time = datetime.fromisoformat(self.expected_arrival_time)
        current_localized_time = TimeUtils.get_current_localized_time()
        self.arriving_in_dict = TimeUtils.compute_time_diff(current_localized_time, self.expected_arrival_time)


    def __str__(self):
        return f'{self.arriving_in_dict}'

        

@dataclass_json(letter_case=LetterCase.CAMEL)
@dataclass
class PointPassingTimes:
    passing_times: List[PassingTime] = field(default_factory=list)
    point_id: str = ""



In [22]:
legrand_stop_id = "1047F"
bearer = '2753702fdd1e39b1bf9526569c578292'
response = make_api_request(legrand_stop_id, bearer)
response

{'points': [{'passingTimes': [{'destination': {'fr': 'ROODEBEEK',
      'nl': 'ROODEBEEK'},
     'expectedArrivalTime': '2020-04-06T16:09:00+02:00',
     'lineId': '8'},
    {'destination': {'fr': 'LEGRAND', 'nl': 'LEGRAND'},
     'expectedArrivalTime': '2020-04-06T16:10:00+02:00',
     'lineId': '93',
     'message': {'fr': 'NE PAS EMBARQUER', 'nl': 'NIET INSTAPPEN'}},
    {'destination': {'fr': 'ROODEBEEK', 'nl': 'ROODEBEEK'},
     'expectedArrivalTime': '2020-04-06T16:23:00+02:00',
     'lineId': '8'},
    {'destination': {'fr': 'LEGRAND', 'nl': 'LEGRAND'},
     'expectedArrivalTime': '2020-04-06T16:23:00+02:00',
     'lineId': '93',
     'message': {'fr': 'NE PAS EMBARQUER', 'nl': 'NIET INSTAPPEN'}}],
   'pointId': '1047'}]}

In [23]:
passages = PointPassingTimes.schema().load(response['points'], many=True) 

passages

[PointPassingTimes(passing_times=[PassingTime(destination=Destination(fr='ROODEBEEK', nl='ROODEBEEK'), message=None, line_id='8', expected_arrival_time=datetime.datetime(2020, 4, 6, 16, 9, tzinfo=datetime.timezone(datetime.timedelta(seconds=7200)))), PassingTime(destination=Destination(fr='LEGRAND', nl='LEGRAND'), message=Message(fr='NE PAS EMBARQUER', nl='NIET INSTAPPEN'), line_id='93', expected_arrival_time=datetime.datetime(2020, 4, 6, 16, 10, tzinfo=datetime.timezone(datetime.timedelta(seconds=7200)))), PassingTime(destination=Destination(fr='ROODEBEEK', nl='ROODEBEEK'), message=None, line_id='8', expected_arrival_time=datetime.datetime(2020, 4, 6, 16, 23, tzinfo=datetime.timezone(datetime.timedelta(seconds=7200)))), PassingTime(destination=Destination(fr='LEGRAND', nl='LEGRAND'), message=Message(fr='NE PAS EMBARQUER', nl='NIET INSTAPPEN'), line_id='93', expected_arrival_time=datetime.datetime(2020, 4, 6, 16, 23, tzinfo=datetime.timezone(datetime.timedelta(seconds=7200))))], point_

In [24]:
expected_arrival_times = parse_waiting_times(response, 'ROODEBEEK' )

current_time_utc = datetime.utcnow()
current_time_be = pytz.utc.localize(current_time_utc)
tram_arrival_time = expected_arrival_times[0]

waiting_time = compute_time_diff(current_time_be, tram_arrival_time)
print(waiting_time)


['2020-04-06T16:09:00+02:00', '2020-04-06T16:23:00+02:00']
0:02:40.417571


In [25]:
def get_shapefiles():
    open_data_api_endpoint = "https://opendata-api.stib-mivb.be/Files/2.0/Shapefiles" + stop_id
    headers = {"Accept": "application/json"}

    response = requests.get(open_data_api_endpoint, auth=BearerAuth(bearer))
    
    return response.json()

In [26]:
from enum import Enum

class RouteType(Enum):
    TRAMWAY = 0
    METRO = 1
    BUS = 3

In [27]:
ROUTE_DATAPATH = "gtfs/routes.txt"
STOP_DATAPATH = "gtfs/stops.txt"
routes_df = pd.read_csv(ROUTE_DATAPATH)
stops_df = pd.read_csv(STOP_DATAPATH)

In [28]:
@dataclass_json(letter_case=LetterCase.CAMEL)
@dataclass
class LinePoint:
    id: str = ""
    order: int = 0
    stop_name: str = ""

    def __post_init__(self):
        try:
            self.stop_name = stops_df[stops_df["stop_id"] == self.id].iloc[0]["stop_name"]
        except:
            self.stop_name = ""
            


@dataclass_json(letter_case=LetterCase.CAMEL)
@dataclass
class LineDetails:
    destination: Optional[Destination] = None
    direction: str = ""
    line_id: str = ""
    points: List[LinePoint] = field(default_factory=list)
    route_type: Optional[RouteType] = None

    def __post_init__(self):
        self.route_type = RouteType(
            routes_df[routes_df["route_short_name"] == self.line_id].iloc[0][
                "route_type"
            ]
        )


In [29]:
def make_line_info_api_request(line_id, bearer):
    open_data_api_endpoint = "https://opendata-api.stib-mivb.be/NetworkDescription/1.0/PointByLine/" + line_id
    headers = {"Accept": "application/json"}

    response = requests.get(open_data_api_endpoint, auth=BearerAuth(bearer))
    
    return response.json()


In [30]:
line_info_api_response = make_line_info_api_request("93", bearer)
line_info_api_response

{'lines': [{'destination': {'fr': 'LEGRAND', 'nl': 'LEGRAND'},
   'direction': 'Suburb',
   'lineId': '93',
   'points': [{'id': '5400', 'order': 1},
    {'id': '5402', 'order': 2},
    {'id': '5399F', 'order': 3},
    {'id': '5403', 'order': 4},
    {'id': '5010', 'order': 5},
    {'id': '6501', 'order': 6},
    {'id': '6505', 'order': 7},
    {'id': '6504', 'order': 8},
    {'id': '4130F', 'order': 9},
    {'id': '6100', 'order': 10},
    {'id': '6122', 'order': 11},
    {'id': '1073', 'order': 12},
    {'id': '6103F', 'order': 13},
    {'id': '6178', 'order': 14},
    {'id': '5762', 'order': 15},
    {'id': '1646F', 'order': 16},
    {'id': '6012G', 'order': 17},
    {'id': '6412F', 'order': 18},
    {'id': '6014', 'order': 19},
    {'id': '6437F', 'order': 20},
    {'id': '6308F', 'order': 21},
    {'id': '6309F', 'order': 22},
    {'id': '6310F', 'order': 23},
    {'id': '6311', 'order': 24},
    {'id': '6312', 'order': 25},
    {'id': '6313', 'order': 26},
    {'id': '6314', 'ord

In [31]:
lines = LineDetails.schema().load(line_info_api_response['lines'], many=True) 

In [32]:
line_details_serialized = [line.to_dict() for line in lines]
line_details_serialized

[{'destination': {'fr': 'LEGRAND', 'nl': 'LEGRAND'},
  'direction': 'Suburb',
  'lineId': '93',
  'points': [{'id': '5400', 'order': 1, 'stopName': ''},
   {'id': '5402', 'order': 2, 'stopName': 'STIENON'},
   {'id': '5399F', 'order': 3, 'stopName': 'POLYCL. BRUGMANN'},
   {'id': '5403', 'order': 4, 'stopName': 'HOPITAL BRUGMANN'},
   {'id': '5010', 'order': 5, 'stopName': 'GUILL. DE GREEF'},
   {'id': '6501', 'order': 6, 'stopName': ''},
   {'id': '6505', 'order': 7, 'stopName': ''},
   {'id': '6504', 'order': 8, 'stopName': ''},
   {'id': '4130F', 'order': 9, 'stopName': 'BOCKSTAEL'},
   {'id': '6100', 'order': 10, 'stopName': 'MOORSLEDE'},
   {'id': '6122', 'order': 11, 'stopName': 'PRINC. CLEMENTINE'},
   {'id': '1073', 'order': 12, 'stopName': 'OUTRE-PONTS'},
   {'id': '6103F', 'order': 13, 'stopName': 'JULES DE TROOZ'},
   {'id': '6178', 'order': 14, 'stopName': 'MASUI'},
   {'id': '5762', 'order': 15, 'stopName': 'THOMAS'},
   {'id': '1646F', 'order': 16, 'stopName': 'LIEDTS'},


In [33]:
slots = {'destination_name': {'confirmation_status': 'NONE',
'name': 'destination_name',
'resolutions': {'resolutions_per_authority': [{'authority': 'amzn1.er-authority.echo-sdk.amzn1.ask.skill.789c381d-5f2c-469e-a888-ee60e260c9de.DESTINATION_NAME',
'status': {'code': 'ER_SUCCESS_NO_MATCH'},
'values': None},
{'authority': 'amzn1.er-authority.echo-sdk.dynamic.amzn1.ask.skill.789c381d-5f2c-469e-a888-ee60e260c9de.DESTINATION_NAME',
'status': {'code': 'ER_SUCCESS_MATCH'},
'values': [{'value': {'id': 'VANDERKINDERE',
'name': 'VANDERKINDERE'}}]}]},
'value': 'vanderkindere'}, 'stop_name': {'confirmation_status': 'NONE',
'name': 'stop_name',
'resolutions': {'resolutions_per_authority': [{'authority': 'amzn1.er-authority.echo-sdk.amzn1.ask.skill.789c381d-5f2c-469e-a888-ee60e260c9de.STOP_NAME',
'status': {'code': 'ER_SUCCESS_MATCH'},
'values': [{'value': {'id': '1059',
'name': 'LEGRAND'}}]},
{'authority': 'amzn1.er-authority.echo-sdk.dynamic.amzn1.ask.skill.789c381d-5f2c-469e-a888-ee60e260c9de.STOP_NAME',
'status': {'code': 'ER_SUCCESS_MATCH'},
'values': [{'value': {'id': '1048F',
'name': 'LEGRAND'}},
{'value': {'id': '1049F',
'name': 'LEGRAND'}}]}]},
'value': 'le legrand'}}

In [129]:
def parse_successful_entity_resolution_results_for_slot(slot):
    entity_resolutions ={"values": [], "resolved": ''}
    entity_resolutions["resolved"] = slot["value"]
    success_entity_resolutions= [resolution_per_authority["values"]
                                        for resolution_per_authority in 
                                slot["resolutions"]["resolutions_per_authority"] 
                                        if resolution_per_authority["status"]["code"]=="ER_SUCCESS_MATCH"]
    flatten_success_entity_resolutions = [item for sublist in success_entity_resolutions for item in sublist]
    entity_resolutions["values"] = flatten_success_entity_resolutions
    return entity_resolutions

In [167]:
def filter_correct_slot_value(success_slot_er_results,line_details_serialized, destination):
    correct_destination_line_details = list(filter(lambda line_detail: line_detail["destination"]["fr"] == destination_name, line_details_serialized))[-1]
    plausible_stop_ids = [item["value"]["id"] for item in success_slot_er_results["values"]]
    correct_slot_id = list(filter(lambda point: point["id"] in plausible_stop_ids, correct_destination_line_details["points"]))[-1]
    return correct_slot_id
    
    

In [168]:
success_slot_er_results

{'values': [{'value': {'id': '1059', 'name': 'LEGRAND'}},
  {'value': {'id': '1048F', 'name': 'LEGRAND'}},
  {'value': {'id': '1049F', 'name': 'LEGRAND'}}],
 'resolved': 'le legrand'}

In [169]:
destination_name = 'STADE'
success_slot_er_results = parse_successful_entity_resolution_results_for_slot(slots['stop_name'])
correct_slot_id = filter_correct_slot_value(success_slot_er_results,line_details_serialized,destination_name)
correct_slot_id


{'id': '1059', 'order': 1, 'stopName': 'LEGRAND'}

In [170]:
from ask_sdk_model.er.dynamic import (
    Entity,
    EntityValueAndSynonyms,
    EntityListItem,
    UpdateBehavior,
)

In [171]:
    def _build_entity_list_items_from_line_details(
        line_details: List[LineDetails],
    ) -> List[EntityListItem]:
        """
        Create list of dynamic entity items from line details
        """
        
        points = line_details[0].points + line_details[1].points
        destinations = [line.destination for line in line_details]

        stop_entities = [Entity(id=point.id, name=EntityValueAndSynonyms(value=point.stop_name)) for point in points]
        destination_entities = [Entity(id=destination.fr, name=EntityValueAndSynonyms(value=destination.fr)) for destination in destinations]
        entity_list_items = [
            EntityListItem(name="STOP_NAME", values=stop_entities),
            EntityListItem(name="DESTINATION_NAME", values=destination_entities),
        ]
        return entity_list_items

In [173]:
import uuid
str(uuid.uuid4())

'edfc821d-aa36-486c-a268-da76828317d0'

In [175]:
a = [1]
a[-1]

1